In [2]:
#Mapping São Paulo's districts into a JSON file

In [16]:
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium

print("Imported libraries!")

Imported libraries!


In [17]:
#Getting the wikipedia page
r = requests.get("https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_S%C3%A3o_Paulo_por_popula%C3%A7%C3%A3o").text
soup = BeautifulSoup(r, 'html.parser')

#print(soup.get_text())

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Lista dos distritos de São Paulo por população – Wikipédia, a enciclopédia livre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgMonthNamesShort":["","jan.","fev.","mar.","abr.","mai.","jun.","jul.","ago.","set.","out.","nov.","dez."],"wgRequestId":"XcmJIgpAMFIAACXB0fMAAACG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Lista_dos_distritos_de_São_Paulo_por_população","wgTitle":"Lista dos distritos de São Paulo por população","wgCurRevisionId":56123564,"wgRevisionId":56123564,"wgArticleId":2953558,"wgIsArticle":!0,"wgIsRedirect":!

In [18]:
#Mapping the table's columns into a list
ind = []
dist = []
pop = []

soup.find('table').find_all('tr')
soup.find('table').find_all('tr')
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        ind.append(cells[0].text.rstrip('\n'))
        dist.append(cells[1].text.rstrip('\n'))
        pop.append(cells[2].text.rstrip('\n'))
        
data = {"Districts": dist, "Population": pop}
df = pd.DataFrame(data)
df.index = df.index + 1
df.head(10)
        
#ind, dist, pop

,Districts,Population
1,Grajaú,444.593
2,Sapopemba,296.042
3,Jardim Ângela,291.798
4,Brasilândia,280.069
5,Capão Redondo,275.230
6,Jardim São Luís,261.586
7,Cidade Ademar,248.215
8,Itaim Paulista,241.026
9,Sacomã,237.769
10,Jaraguá,220.292


In [21]:
#Using Nominatim to get the district's Latitudes and Longitudes
Latitude = []
Longitude = []

for distrito in df['Districts']:
    
    geolocator = Nominatim(user_agent="github.com/caiobsilva", timeout = 3)
    location = geolocator.geocode(distrito + ", São Paulo, SP, Brazil")
    latitude = location.latitude
    longitude = location.longitude

    print(latitude, longitude)
    
    Latitude.append(latitude)
    Longitude.append(longitude)

-23.78579755 -46.6655745956966
-23.6043265 -46.5098851
-23.7122455 -46.7712056726464
-23.4482715 -46.6902692709221
-23.6588586 -46.7677529
-23.68357325 -46.737762089372
-23.6730116 -46.6552806
-23.5017648 -46.3996091
-23.6012617 -46.6025821
-23.455682 -46.7395122
-23.5824973 -46.4092065
-23.6488796 -46.7587289
-23.6520656 -46.650037329077
-23.7139643 -46.6990882
-23.5360799 -46.4555099
-22.95714 -45.547526
-23.53624775 -46.4100218474088
-23.500294 -46.4587173520583
-23.485533 -46.7219385
-23.510151 -46.4178927840909
-22.741347 -46.894846
-23.5982995 -46.4817046
-23.4493375 -46.6636363725781
-23.48228355 -46.4234102269825
-23.5058996 -46.5314253
-23.627159 -46.4532406421398
-23.82479075 -46.7330778497018
-23.4874636 -46.6951317
-23.594946 -46.5458997982925
-23.5627696 -46.4943334
-23.487707 -46.5844955
-23.6182705 -46.4190279394861
-23.523683 -46.5437815
-23.4916735 -46.4840701436689
-23.5797919 -46.769639
-23.58919 -46.6346528
-23.5020677 -46.6247204
-23.54230815 -46.4156048017275
-23.

In [22]:
#Adding Latitudes and Longitudes to the df
df['Longitude'] = Longitude
df['Latitude'] = Latitude

df.head()

,Districts,Population,Longitude,Latitude
1,Grajaú,444.593,-46.665575,-23.785798
2,Sapopemba,296.042,-46.509885,-23.604326
3,Jardim Ângela,291.798,-46.771206,-23.712246
4,Brasilândia,280.069,-46.690269,-23.448272
5,Capão Redondo,275.230,-46.767753,-23.658859


In [23]:
#Converting the dataframe into a JSON file
with open('sp_dist.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False)

In [14]:
#Showing results on a map through Folium
map = folium.Map(location=[-23.5489, -46.638823], zoom_start=10.2)

for lat, lng, dist, pop in zip(df['Latitude'], df['Longitude'], df['Distritos'], df['População']):
    label = 'Distrito: {}, População: {}'.format(dist, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

KeyError: 'Latitude'